In [2]:
%run packages_imported.py
%run EDA.py

In [3]:
#df_all_news_processed = pd.read_pickle(r'data/original_data/pickles/df_all_news_processed_post_dataset_distribution')

In [20]:
frame1 = df_all_news_processed.loc[df_all_news_processed['Category'] == 'entertainment'][:1000]
frame2 = df_all_news_processed.loc[df_all_news_processed['Category'] == 'financial'][:1000]
frame3 = df_all_news_processed.loc[df_all_news_processed['Category'] == 'political'][:1000]
frame4 = df_all_news_processed.loc[df_all_news_processed['Category'] == 'sport'][:1000]
frame5 = df_all_news_processed.loc[df_all_news_processed['Category'] == 'technology'][:1000]
frame6 = df_all_news_processed.loc[df_all_news_processed['Category'] == 'world'][:1000]
frame7 = df_all_news_processed.loc[df_all_news_processed['Category'] == 'travel'][:1000]

frames = [frame1, frame2, frame3, frame4, frame5, frame6, frame7]
df_processed_smaller = pd.concat(frames)
df_processed_smaller.reset_index()
df_processed_smaller.to_pickle('data/original_data/pickles/df_processed_smaller_before_tuning')

--------

In [3]:
df_all_news_processed = pd.read_pickle(r'data/original_data/pickles/df_processed_smaller_before_tuning')

1398056

In [4]:
t1 = time.time() 

from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7)
X = tfidfconverter.fit_transform(df_all_news_processed['Text']).toarray()

EDA.process_time(round(time.time()-t1))

Process completed.
Time taken: 2 seconds


In [113]:
from sklearn.model_selection import GridSearchCV

In [114]:
t1 = time.time() 

y = df_all_news_processed['Category']

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

r_f_c = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, random_state=0)
param_grid = { 
    'n_estimators': [200, 700],
    'max_features': ['auto', 'log2']
}
CV_rfc = GridSearchCV(estimator=r_f_c, param_grid=param_grid, cv= 5)

CV_rfc.fit(X_train, y_train) 
print (CV_rfc.best_params_)
EDA.process_time(round(time.time()-t1))

{'max_features': 'log2', 'n_estimators': 700}
Process completed.
Time taken: 2mins 11secs


## Naive Bayes (Multinomial?)

In [13]:
from sklearn.model_selection import train_test_split

#X = tfidfconverter.fit_transform(df_all_news_processed['Text']).toarray()
X = df_all_news_processed['Text']
y = df_all_news_processed['Category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)


In [19]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB())])

In [43]:
nb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [44]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
y_pred = nb.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))
my_tags = y_train.drop_duplicates()
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.6642857142857143
               precision    recall  f1-score   support

    financial       0.69      0.81      0.74       203
       travel       0.64      0.78      0.70       224
        sport       0.69      0.40      0.51       178
    political       0.82      0.67      0.74       215
entertainment       0.54      0.66      0.59       183
        world       0.69      0.72      0.70       186
   technology       0.63      0.58      0.60       211

     accuracy                           0.66      1400
    macro avg       0.67      0.66      0.66      1400
 weighted avg       0.67      0.66      0.66      1400



In [45]:
nb.score(X_test, y_test)

0.6642857142857143

## Random Forest

In [37]:
#type(y_train)
y_train.drop_duplicates()

32167         financial
158051           travel
95851             sport
64923         political
931       entertainment
190190            world
127249       technology
Name: Category, dtype: object

## XGBoost

## Support Vector Machine (SVM)

## Logistic Regression

## K Nearest Neighbour (?)